In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score,precision_score,precision_score,recall_score,precision_recall_curve,auc,classification_report
from sklearn.linear_model import LogisticRegressionCV,LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split,GridSearchCV

In [2]:
train_features = pd.read_csv('training_set_values.csv')
train_labels = pd.read_csv('training_set_labels.csv')
test_features = pd.read_csv('test_set_values.csv')
df = pd.merge(train_features,train_labels,on='id')


In [3]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [4]:
df.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'status_group'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [6]:
df.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [7]:
df.isna().any()[df.isna().any() == True]

funder               True
installer            True
subvillage           True
public_meeting       True
scheme_management    True
scheme_name          True
permit               True
dtype: bool

In [8]:
df.isna().mean()[df.isna().mean() > 0]

funder               0.061195
installer            0.061532
subvillage           0.006246
public_meeting       0.056128
scheme_management    0.065269
scheme_name          0.474175
permit               0.051448
dtype: float64

The Scheme_Name columns has about 47% missing data. We will drop this column.

We will also drop the null rows of the other columns since they consist of a small percentage of the data (<7%).

In [ ]:
df.drop(axis=1,columns=['scheme_name'],inplace=True)

In [10]:
df.dropna(inplace=True)

We will confirm that the null values have been dropped

In [11]:
df.isna().mean()[df.isna().mean() > 0]

Series([], dtype: float64)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48288 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     48288 non-null  int64  
 1   amount_tsh             48288 non-null  float64
 2   date_recorded          48288 non-null  object 
 3   funder                 48288 non-null  object 
 4   gps_height             48288 non-null  int64  
 5   installer              48288 non-null  object 
 6   longitude              48288 non-null  float64
 7   latitude               48288 non-null  float64
 8   wpt_name               48288 non-null  object 
 9   num_private            48288 non-null  int64  
 10  basin                  48288 non-null  object 
 11  subvillage             48288 non-null  object 
 12  region                 48288 non-null  object 
 13  region_code            48288 non-null  int64  
 14  district_code          48288 non-null  int64  
 15  lg

In [13]:
df.select_dtypes('number')

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
0,69572,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999
2,34310,25.0,686,37.460664,-3.821329,0,21,4,250,2009
3,67743,0.0,263,38.486161,-11.155298,0,90,63,58,1986
5,9944,20.0,0,39.172796,-4.765587,0,4,8,1,2009
6,19816,0.0,0,33.362410,-3.766365,0,17,3,0,0
...,...,...,...,...,...,...,...,...,...,...
59394,11164,500.0,351,37.634053,-6.124830,0,5,6,89,2007
59395,60739,10.0,1210,37.169807,-3.253847,0,3,5,125,1999
59396,27263,4700.0,1212,35.249991,-9.070629,0,11,4,56,1996
59398,31282,0.0,0,35.861315,-6.378573,0,1,4,0,0


In [14]:
df.select_dtypes('object')

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,2011-03-14,Roman,Roman,none,Lake Nyasa,Mnyusi B,Iringa,Ludewa,Mundindi,True,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
2,2013-02-25,Lottery Club,World vision,Kwa Mahundi,Pangani,Majengo,Manyara,Simanjiro,Ngorika,True,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,2013-01-28,Unicef,UNICEF,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mahakamani,Mtwara,Nanyumbu,Nanyumbu,True,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
5,2011-03-13,Mkinga Distric Coun,DWE,Tajiri,Pangani,Moa/Mwereme,Tanga,Mkinga,Moa,True,...,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,functional
6,2012-10-01,Dwsp,DWSP,Kwa Ngomho,Internal,Ishinabulandi,Shinyanga,Shinyanga Rural,Samuye,True,...,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,non functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59394,2011-03-09,World Bank,ML appro,Chimeredya,Wami / Ruvu,Komstari,Morogoro,Mvomero,Diongoya,True,...,soft,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,non functional
59395,2013-05-03,Germany Republi,CES,Area Three Namba 27,Pangani,Kiduruni,Kilimanjaro,Hai,Masama Magharibi,True,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
59396,2011-05-07,Cefa-njombe,Cefa,Kwa Yahona Kuvala,Rufiji,Igumbilo,Iringa,Njombe,Ikondo,True,...,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional
59398,2011-03-08,Malec,Musa,Mshoro,Rufiji,Mwinyi,Dodoma,Chamwino,Mvumi Makulu,True,...,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional


In [15]:
df.select_dtypes('object').columns

Index(['date_recorded', 'funder', 'installer', 'wpt_name', 'basin',
       'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'recorded_by',
       'scheme_management', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group',
       'status_group'],
      dtype='object')

We will add a column called Year Recorded.

We will then drop the Date_Recorded column

In [ ]:
df['Year_Recorded'] = df['date_recorded'].str.slice(0,4)

In [17]:
df[['date_recorded','Year_Recorded']]

,date_recorded,Year_Recorded
0,2011-03-14,2011
2,2013-02-25,2013
3,2013-01-28,2013
5,2011-03-13,2011
6,2012-10-01,2012
...,...,...
59394,2011-03-09,2011
59395,2013-05-03,2013
59396,2011-05-07,2011
59398,2011-03-08,2011


In [34]:
df['Year_Recorded'].astype('int64')

0        2011
2        2013
3        2013
5        2011
6        2012
         ... 
59394    2011
59395    2013
59396    2011
59398    2011
59399    2011
Name: Year_Recorded, Length: 48288, dtype: int64

In [35]:
df['Year_Recorded'].unique()

array(['2011', '2013', '2012', '2004', '2002'], dtype=object)

We will now drop the date_recorded column

In [ ]:
df.drop(axis=1,columns=['date_recorded'],inplace=True)

In [18]:
df.funder.value_counts()

Government Of Tanzania    8080
Danida                    2920
Hesawa                    1388
Kkkt                      1260
World Bank                1086
                          ... 
Tove                         1
Unp/aict                     1
Redap                        1
Wbk                          1
Aixos                        1
Name: funder, Length: 1586, dtype: int64

In [19]:
df.installer.value_counts()

DWE           14637
Government     1438
RWE            1031
Commu           893
KKKT            885
              ...  
EGYPT             1
Kando             1
mbeje             1
Mr Kas            1
USAID             1
Name: installer, Length: 1787, dtype: int64

In [20]:
df['source'].unique()

array(['spring', 'dam', 'machine dbh', 'other', 'shallow well', 'river',
       'hand dtw', 'rainwater harvesting', 'lake', 'unknown'],
      dtype=object)

In [21]:
df['source_type'].unique()

array(['spring', 'dam', 'borehole', 'other', 'shallow well', 'river/lake',
       'rainwater harvesting'], dtype=object)

In [22]:
df[['source','source_type']].value_counts()

source                source_type         
spring                spring                  14852
shallow well          shallow well            12415
machine dbh           borehole                 8730
river                 river/lake               8506
rainwater harvesting  rainwater harvesting     1607
hand dtw              borehole                  779
lake                  river/lake                604
dam                   dam                       581
other                 other                     182
unknown               other                      32
dtype: int64

The source and source_type columns give almost the exact same information.

'Machine dbh' and 'Hand dtw' both are borehole source types.

Source has a few more unique values so we will drop the source_type column.

In [ ]:
df.drop(axis=1,columns='source',inplace=True)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,Year_Recorded
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,groundwater,communal standpipe,communal standpipe,functional,2011
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,surface,communal standpipe multiple,communal standpipe,functional,2013
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,2013
5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Tajiri,0,...,salty,salty,enough,enough,other,unknown,communal standpipe multiple,communal standpipe,functional,2011
6,19816,0.0,2012-10-01,Dwsp,0,DWSP,33.362410,-3.766365,Kwa Ngomho,0,...,soft,good,enough,enough,borehole,groundwater,hand pump,hand pump,non functional,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59394,11164,500.0,2011-03-09,World Bank,351,ML appro,37.634053,-6.124830,Chimeredya,0,...,soft,good,enough,enough,borehole,groundwater,communal standpipe,communal standpipe,non functional,2011
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,soft,good,enough,enough,spring,groundwater,communal standpipe,communal standpipe,functional,2013
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,soft,good,enough,enough,river/lake,surface,communal standpipe,communal standpipe,functional,2011
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,soft,good,insufficient,insufficient,shallow well,groundwater,hand pump,hand pump,functional,2011


In [23]:
df['waterpoint_type'].unique()

array(['communal standpipe', 'communal standpipe multiple', 'hand pump',
       'other', 'improved spring', 'cattle trough', 'dam'], dtype=object)

In [24]:
df['waterpoint_type_group'].unique()

array(['communal standpipe', 'hand pump', 'other', 'improved spring',
       'cattle trough', 'dam'], dtype=object)

In [25]:
df[['waterpoint_type_group','waterpoint_type']].value_counts()

waterpoint_type_group  waterpoint_type            
communal standpipe     communal standpipe             23837
hand pump              hand pump                      13602
communal standpipe     communal standpipe multiple     5459
other                  other                           4651
improved spring        improved spring                  651
cattle trough          cattle trough                     82
dam                    dam                                6
dtype: int64

The waterpoint_type and the water_point_type_group columns give us the same information.

The waterpoint_type column has a few more unique values.

We will drop the waterpoint_type_group column

In [ ]:
df.drop(axis=1,columns='water_type_group',inplace=True)

In [26]:
df['quantity'].unique()

array(['enough', 'dry', 'seasonal', 'insufficient', 'unknown'],
      dtype=object)

In [27]:
df['quantity_group'].unique()

array(['enough', 'dry', 'seasonal', 'insufficient', 'unknown'],
      dtype=object)

In [28]:
df[['quantity','quantity_group']].value_counts()

quantity      quantity_group
enough        enough            28355
insufficient  insufficient      11799
dry           dry                4945
seasonal      seasonal           2899
unknown       unknown             290
dtype: int64

The two columns give you the same information. 

So we will drop the quantity column

In [ ]:
df.drop(axis=1,columns='quantity',inplace=True)

In [29]:
df[['extraction_type','extraction_type_group','extraction_type_class']].value_counts()

extraction_type            extraction_type_group  extraction_type_class
gravity                    gravity                gravity                  23036
nira/tanira                nira/tanira            handpump                  6260
other                      other                  other                     4538
submersible                submersible            submersible               3645
swn 80                     swn 80                 handpump                  2858
mono                       mono                   motorpump                 2382
india mark ii              india mark ii          handpump                  2047
afridev                    afridev                handpump                  1346
ksb                        submersible            submersible               1330
other - rope pump          rope pump              rope pump                  212
other - swn 81             other handpump         handpump                   199
windmill                   wind-power

I will keep the extraction_type and extraction_type_class columns. 

We will get all the information from these two columns.

They do not have an order so we will use OneHotEncoding

In [ ]:
df.drop(axis=1,columns='extraction_type_group',inplace=True)

In [30]:
df['management'].unique()

array(['vwc', 'private operator', 'wug', 'water board', 'wua', 'company',
       'other', 'water authority', 'parastatal', 'other - school',
       'unknown', 'trust'], dtype=object)

In [31]:
df['management_group'].unique()

array(['user-group', 'commercial', 'other', 'parastatal', 'unknown'],
      dtype=object)

In [32]:
df[['management','management_group']].value_counts()

management        management_group
vwc               user-group          33577
wug               user-group           4807
water board       user-group           2665
wua               user-group           2288
private operator  commercial           1479
parastatal        parastatal           1331
water authority   commercial            790
company           commercial            654
other             other                 449
other - school    other                  99
trust             commercial             75
unknown           unknown                74
dtype: int64

Management and management_group columns are giving us different information.

The management_group column seems to tell us the category that the entry in management is in.

I will keep both the columns.

In [38]:
df['payment'].unique()

array(['pay annually', 'pay per bucket', 'never pay',
       'pay when scheme fails', 'other', 'pay monthly', 'unknown'],
      dtype=object)

In [39]:
df['payment_type'].unique()

array(['annually', 'per bucket', 'never pay', 'on failure', 'other',
       'monthly', 'unknown'], dtype=object)

We will ordinally encode the quantity column.

We might have to drop the unknown rows.

We will have to check how many of the rows are part of the unknown category.

In [37]:
df['quantity'].unique()

array(['enough', 'dry', 'seasonal', 'insufficient', 'unknown'],
      dtype=object)

In [40]:
df['quantity'].value_counts()

enough          28355
insufficient    11799
dry              4945
seasonal         2899
unknown           290
Name: quantity, dtype: int64

In [41]:
df['water_quality'].unique()

array(['soft', 'salty', 'unknown', 'milky', 'fluoride', 'coloured',
       'salty abandoned', 'fluoride abandoned'], dtype=object)

In [42]:
df['quality_group'].unique()

array(['good', 'salty', 'unknown', 'milky', 'fluoride', 'colored'],
      dtype=object)